<a href="https://colab.research.google.com/github/MarcelloCeresini/ChessBreaker/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
username = 'MarcelloCeresini'
repository = 'ChessBreaker'

In [3]:
# COLAB ONLY CELLS
try:
    import google.colab
    IN_COLAB = True
    !nvidia-smi             # Check which GPU has been chosen for us
    !rm -rf logs
    #from google.colab import drive
    #drive.mount('/content/drive')
    #%cd /content/drive/MyDrive/GitHub/
    !git clone https://github.com/{username}/{repository}.git
    %cd {repository}
    %ls
    !pip3 install anytree
except:
    IN_COLAB = False

In [4]:
import numpy as np
import tensorflow as tf
import chess
from anytree import Node

import utils
from utils import plane_dict, Config, x_y_from_position

conf = Config()
board = chess.Board()

legal_moves = board.legal_moves
for move in legal_moves:
    print(move.uci())  
print(legal_moves)


g1h3
g1f3
b1c3
b1a3
h2h3
g2g3
f2f3
e2e3
d2d3
c2c3
b2b3
a2a3
h2h4
g2g4
f2f4
e2e4
d2d4
c2c4
b2b4
a2a4
<LegalMoveGenerator at 0x7f577fba0130 (Nh3, Nf3, Nc3, Na3, h3, g3, f3, e3, d3, c3, b3, a3, h4, g4, f4, e4, d4, c4, b4, a4)>


In [5]:
# print(board.fen())
print(list(board.pieces(1,True)))

[8, 9, 10, 11, 12, 13, 14, 15]


In [72]:
def uniform_tensor(x):
    return tf.fill(conf.BOARD_SHAPE, x)

EMPTY_PLANES = tf.zeros([*conf.BOARD_SHAPE, conf.TOTAL_PLANES], dtype=conf.PLANES_DTYPE)

def special_input_planes(board):
    return tf.transpose(tf.vectorized_map( 
            uniform_tensor,
            tf.constant([
                int(board.turn                                 ),
                int(board.fullmove_number-1                    ),   # don't know why but it starts from 1 on move 1, just reduce it by one and now it's right  
                int(board.has_kingside_castling_rights(True)   ),   # True for White
                int(board.has_queenside_castling_rights(True)  ),
                int(board.has_kingside_castling_rights(False)  ),   # False for Black
                int(board.has_queenside_castling_rights(False) ),
                int(board.halfmove_clock                       )
            ], dtype=conf.PLANES_DTYPE)
        ), [1,2,0])


def update_planes(current, board, board_history):

    if current == None: # root, initialize to zero
        current = tf.zeros([*conf.BOARD_SHAPE, conf.PAST_TIMESTEPS*conf.REPEATED_PLANES], dtype=conf.PLANES_DTYPE)
    
    planes = [] # since we cannot "change" a tensor after creating it, we create them one by one in a list and then stack them

    for color in range(2):                                                                  # for each color
        for piece_type in range(1, conf.N_PIECE_TYPES+1):                                   # for each piece type
            indices = []                                                                    # we save the position on the board in a list
            for position in list(board.pieces(piece_type, color)):                          # for each piece of that type
                indices.append(x_y_from_position(position))                                 # the function transforms a number (1-64) into a tuple (1-8, 1-8)
            values = np.array([1]*len(indices), dtype=conf.PLANES_DTYPE_NP)                 # simply "1" in a list with unit8 dtype
            tensor = tf.sparse.to_dense(tf.SparseTensor(dense_shape=[*conf.BOARD_SHAPE], indices=indices, values=values))   ### created as sparse because it's easier, needed as dense afterwards
            planes.append(tensor)
        planes.append(uniform_tensor(tf.constant(board_history.count(board_history[-1]), dtype=conf.PLANES_DTYPE)))      # adding a "repetition plane" for each color (simply count how many times the current (last) position has been encountered)

    current_planes = tf.transpose(tf.stack(planes), [1,2,0])                                # transpose them to have the planes as last dimension

    old_planes = tf.slice(current, begin=[0,0,0], size=[*conf.BOARD_SHAPE, (conf.PAST_TIMESTEPS-1)*conf.REPEATED_PLANES]) # take the first 7 repetitions, slice them and paste them at the end of the new planes
    
    return tf.concat([current_planes, old_planes, special_input_planes(board)], axis=-1)    # also concat the special planes


In [ ]:
class MyNode(Node): # subclassing Node from Anytree to add some methods

    def update_action_value(self, new_action_value):
        self.action_value += (new_action_value-self.action_value)/(self.visit_count+1)

    def calculate_upper_confidence_bound(self, num_total_iterations=1):
        return self.action_value + conf.expl_param(num_total_iterations)*self.prior/(1+self.visit_count)

    def calculate_move_probability(self, num_total_iterations=1):
        return self.visit_count**(1/conf.temp_param(num_total_iterations))


def MTCS(model, root_node, max_depth, num_restarts):
    
    for i in range(num_restarts):

        while root_node.depth <= max_depth:
            assert root_node.depth >= 0 and root_node.depth <= max_depth, "depth is wrong"

            if root_node.is_leaf:

                full_moves, outcome = model(root_node.planes) # TODO: actually implement good output from model
                priors = tf.boolean_mask(full_moves, utils.mask_moves(legal_moves))

                root_node.action_value = outcome

                for move, prior in zip(legal_moves, priors):
                    new_board = root_node.board.push(move)
                    new_board_history = root_node.board_history.copy()
                    new_board_history.append(new_board.fen()[:-6])
                    MyNode(
                        move, 
                        parent = root_node, 
                        prior = prior,
                        visit_count = 1,
                        board_history = new_board_history,
                        board = new_board, 
                        planes = update_planes(root_node.planes, new_board, new_board_history)
                    )

            if root_node.depth < max_depth:
                children = root_node.children
                values = [child.caclulate_upper_confidence_bound() for child in children]
                root_node = children[np.argmax(values)]
                root_node.visit_count += 1

            else:    
                outcome = root_node.action_value # needed for when depth=max_depth AND NOT LEAF (that means, already visited leaf) --> don't REDO the evaluation, it would give the same result, simply copy it from before
                break                            # no need to descend the tree further, max depth is reached
           
        # barckpropagation of action value through the tree
        while root_node.parent != None:
            assert root_node.depth >= 0 and root_node.depth <= max_depth, "depth is wrong"
            root_node = root_node.parent
            root_node.update_action_value(outcome)
     
    return root_node


def choose_move(root_node):
    children = root_node.children
    assert root_node.children != [], "No children, cannot choose move"

    root_node = np.random.choice(
        children, 
        p=[child.calculate_move_probability() for child in children]    # choose the child proportionally to the number of times it has been visited (exponentiated by a temperature parameter)
    ) 
        
    root_node.parent = None # To detach the subtree and restart with the next move search

    return root_node


def complete_game(board):
    board = chess.Board()
    root_node = MyNode(
        "",                                             # no name needed for initial position
        board = board,
        board_history = board.fen()[:-6],               # we remove the "en passant", "halfmove clock" and "fullmove number" from the fen --> position will be identical even if those values differ
        planes = update_planes(EMPTY_PLANES, board),    # start from empty planes and fill them (usually you need previous planes to fill them)
        action_value=0)

    while not root_node.board.is_game_over(claim_draw=True) and root_node.board.fullmove_number <= conf.MAX_MOVE_COUNT:
        
        root_node = MTCS(current_model, root_node, max_depth = conf.MAX_DEPTH, num_restarts=conf.NUM_RESTARTS)                          # though the root node you can access all the tree
        root_node = choose_move(root_node)

